In [ ]:
# data from https://coinmetrics.io/data-downloads/

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from glob import glob
from pathlib import Path

In [ ]:
%matplotlib inline

In [ ]:
# to fix warning 
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [ ]:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
DAYS = 365  # in days
DATEMIN = '2009-01-03' 
DATEMAX = '2019-07-07'

In [ ]:
df = pd.DataFrame()
for name in glob('data/*.csv'):
    tmp = pd.read_csv(name, 
                      parse_dates=['date'])
    coin = Path(name).stem
    if 'price(USD)' not in tmp.columns:
        print(f'For {coin} price(USD) is missing')
        continue 
    tmp = tmp[['date', 'price(USD)']]
    tmp.columns = ['date', coin]
    
    tmp = tmp.set_index('date')
    tmp = tmp.reindex(pd.date_range(start=DATEMIN, end=DATEMAX, freq='D'))
    df = pd.concat([df, tmp], axis=1, sort=False)   

In [ ]:
coins = df.columns.sort_values()

In [ ]:
df = df.div(df['btc'], axis=0) / df.div(df['btc'], axis=0).shift(DAYS)
df = df.reset_index()

In [ ]:
df_plt = pd.melt(df, 
                 id_vars='index',
                 col_level=0)
df_plt.columns = ['date', 'coin', 'roi']
df_plt.head()

In [ ]:
df_plt.dropna(how='any').to_csv('coins_vs_btc.csv')

In [ ]:
g = sns.FacetGrid(data=df_plt,
                  hue='coin',
                  height=3, 
                  aspect=3, 
                  row='coin',
                  palette='muted',
                  row_order=coins,
                  sharex=False,
                  sharey=False,
                  )
g.map(plt.plot, 'date', 'roi', alpha=.8)

for ax in g.axes.flat:
    ax.axhline(1, ls='--', color='red')

g.set(yscale='log')
g.savefig('figs/vs-btc-not-share-axes.pdf')

In [ ]:
# drop btc for better readability
g = sns.FacetGrid(data=df_plt[df_plt['coin'] != 'btc'].dropna(how='any'),
                  hue="coin",
                  height=3, 
                  aspect=3, 
                  row='coin',
                  palette='muted',
                  row_order=coins.drop('btc'),
                  # sharex=True,
                  # sharey=True,
                  )
g.map(plt.plot, 'date', 'roi', alpha=.8)

for ax in g.axes.flat:
    ax.axhline(1, ls='--', color='red')

g.set(yscale="log")
g.savefig('figs/vs-btc-share-axes.pdf')

In [ ]:
coins.drop('btc')